## Load environment variables

In [93]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

## Model with structured output
### Case: 1

In [103]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    title: str
    actors: list[str] = Field(description="Lead actors names in the movie")

class MovieDetails(BaseModel):
    movieInfo: list[Movie]
    director:str
    budget: float | None = Field(None,description="Budget in rupees")
    year: int = Field(description="Movie release year")

In [104]:
from langchain.chat_models import init_chat_model

model = init_chat_model("groq:qwen/qwen3-32b")
model_with_str = model.with_structured_output(MovieDetails)
model_with_str

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001B6272CE650>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B6272CDBA0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'MovieDetails', 'description': '', 'parameters': {'properties': {'movieInfo': {'items': {'properties': {'title': {'type': 'string'}, 'actors': {'description': 'Lead actors names in the movie', 'items': {'type': 'string'}, 'type': 'array'}}, 'required': ['title', 'actors'], 'type': 'object'}, 'type': 'array'}, 'director': {'type': 'string'}, 'budget': {'anyOf': [{'type': 'number'}, {'type':

#### Output1

In [106]:
import json

response = model_with_str.invoke("Provide the details of the movie RRR")
print(f"MOVIE_INFO:\n{response}")
print(f"\nCAST\n{response.movieInfo}")

MOVIE_INFO:
movieInfo=[Movie(title='RRR', actors=['Ram Charan', 'Jr. NTR', 'Alia Bhatt'])] director='S. S. Rajamouli' budget=500000000.0 year=2022

CAST
[Movie(title='RRR', actors=['Ram Charan', 'Jr. NTR', 'Alia Bhatt'])]


#### Output2

In [110]:
response2 = model_with_str.invoke("Give me the details of a Pushpa 2 movie")
print(response2.model_dump_json(indent=2))

{
  "movieInfo": [
    {
      "title": "Pushpa: The Rule 2",
      "actors": [
        "Allu Arjun",
        "Rashmika Mandanna"
      ]
    }
  ],
  "director": "Sukumar",
  "budget": 450000000.0,
  "year": 2024
}


### Case: 2 

In [111]:
# make minimal changes

class Movie(BaseModel):
    title: str
    actors: list[str] = Field(description="Lead actors names in the movie")   

class MovieDetails(BaseModel):
    movieInfo: list[Movie]
    roles:dict[str,list[str]] = Field(description="Actors and their roles in the movie")

In [114]:
model_with_str2 = model.with_structured_output(MovieDetails, include_raw=True)
model_with_str2

{
  raw: RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001B6272CE650>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B6272CDBA0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'MovieDetails', 'description': '', 'parameters': {'properties': {'movieInfo': {'items': {'properties': {'title': {'type': 'string'}, 'actors': {'description': 'Lead actors names in the movie', 'items': {'type': 'string'}, 'type': 'array'}}, 'required': ['title', 'actors'], 'type': 'object'}, 'type': 'array'}, 'roles': {'additionalProperties': {'items': {'type': 'string'}, 'type': 

#### Output1

In [135]:
response3 = model_with_str2.invoke("Give me the details of a Pushpa movie")
response3

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for details about the movie "Pushpa." Let me see which functions I can use here. The available function is MovieDetails. The parameters for this function include movieInfo, which requires the title and actors, and roles, which are the actors and their roles.\n\nFirst, I need to check if I have the necessary information. Since I don\'t have real-time data, I\'ll have to rely on my existing knowledge. The movie "Pushpa" is a 2021 Indian film starring Allu Arjun in the lead role. The director is Sukumar. Other main actors include Fahadh Faasil and Rashmika Mandanna. \n\nWait, the function requires an array of movieInfo objects, each with title and actors. So I should structure the movieInfo array with the title "Pushpa" and the lead actors. Then, the roles parameter is an object mapping actors to their roles. Allu Arjun plays Pushpa, Fahadh Faasil is Bhanwar, and Rashmika Mandanna is Amrutha. \

#### Output2

In [136]:
print(response3['parsed'].model_dump_json(indent=4))

{
    "movieInfo": [
        {
            "title": "Pushpa",
            "actors": [
                "Allu Arjun",
                "Fahadh Faasil",
                "Rashmika Mandanna"
            ]
        }
    ],
    "roles": {
        "Allu Arjun": [
            "Pushpa"
        ],
        "Fahadh Faasil": [
            "Bhanwar"
        ],
        "Rashmika Mandanna": [
            "Amrutha"
        ]
    }
}


In [137]:
response.keys()

dict_keys(['raw', 'parsed', 'parsing_error'])

#### Explore More

In [78]:
response['raw'].response_metadata

{'token_usage': {'completion_tokens': 305,
  'prompt_tokens': 254,
  'total_tokens': 559,
  'completion_time': 0.579620129,
  'completion_tokens_details': {'reasoning_tokens': 235},
  'prompt_time': 0.010505877,
  'prompt_tokens_details': None,
  'queue_time': 0.056925503,
  'total_time': 0.590126006},
 'model_name': 'qwen/qwen3-32b',
 'system_fingerprint': 'fp_5cf921caa2',
 'service_tier': 'on_demand',
 'finish_reason': 'tool_calls',
 'logprobs': None,
 'model_provider': 'groq'}